<a href="https://colab.research.google.com/github/gabrimatx/Jupyter-Notebooks/blob/main/CUDA_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CUDA Notebook

In [24]:
!nvcc --version
!nvidia-smi
!nvidia-smi --query-gpu=compute_cap --format=csv
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Sun Jan 28 23:06:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  7

## CUDA: Compute Unified Device Architecture
- Enables explicit GPU memory management.
- GPU is viewed as *compute device* and is a co-processor to the CPU, has its own **DRAM** and runs many threads in parallel.
---
- The GPU architecture is built around a scalable array of streaming multiprocessors (SM).
- Every SM in a GPU support the concurrent execution of hundreds of threads.
- Every SM executes thread in groups of 32 called **warps**. All threads in a warp execute the same instruction at the same time.

# Hello world

In [25]:
%%cuda
#include <stdio.h>

__global__ void hello(){
    printf("Hello from block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

int main(){
    hello<<<2, 2>>>();
    cudaDeviceSynchronize();
}

Hello from block: 1, thread: 0
Hello from block: 1, thread: 1
Hello from block: 0, thread: 0
Hello from block: 0, thread: 1

